In [3]:
the_item = "mx5"
max_scrape = 100

insta_path = "explore/tags/"
prefix = "photos"

In [4]:
import base64
from string import ascii_letters, digits, punctuation, printable
from IPython.display import Image, display, HTML
from time import sleep
from collections import Counter
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from copy import copy
import requests
import json
import os
import platform
from datetime import datetime
import numpy as np
import pandas as pd
from ast import literal_eval
from requests import get

In [5]:
!pwd

/home/charly/Desktop/Instagram Scraping


In [6]:
# Create folders as necessary
# Data folder will contain csv, json and html files
data_path = "data/data"  + "_" + prefix + "_" + the_item
# Media folder will contain all the pictures scraped
media_path = "data/media" + "_" + prefix + "_" + the_item

print(data_path)
print(media_path)

try:
    os.mkdir("data")
    print("made new data folder")
except:
    pass

try:
    os.mkdir(media_path)
    print("made new media folder")
except:
    pass

try:
    os.mkdir(data_path)
    print("made new project folder")
except:
    pass

data/data_photos_mx5
data/media_photos_mx5


## Load

In [7]:
# will check if a scrape has already been done for this hashtag
try:
    f = open(data_path+"/metadata.json","r")
    the_posts = json.load(f)
    f.close()
    print("found",len(the_posts),"saved posts.")
except:
    the_posts = {}
    print("This is a new scrape. No saved posts found.")

found 1000 saved posts.


In [8]:
#json configuration file for html rendering. Will extend the json file with each new hashtag
try:
    f = open("config.json","r")
except:
    f = open("default-config.json","r")

conf = json.load(f)
f.close()
if not the_item in conf:
    conf[the_item] = conf["default"]
    f = open("config.json","w")
    json.dump(conf,f)
    f.close()
    print("Updated the config file.")
deets = conf[the_item]

## Set up the web driver

In [160]:
# start the Chrome driver
driver = webdriver.Chrome()

In [162]:
# function to log in on your instagram account
def login_instagram(driver, username, password):
    
    driver.get("https://www.instagram.com/")
    
    Login1 = driver.find_elements_by_class_name("_g9ean")
    for a_element in Login1:
        a_text = a_element.find_element_by_tag_name('a')
    a_text.click()
    
    
    #class_name on the following line will probably change on a daily basis
    input_info = driver.find_elements_by_class_name("_sjplo")
    login_info = []
    for a_element in input_info:
        login_info.append(a_element.find_element_by_tag_name('input'))
    
    login_info[0].send_keys(username)
    login_info[1].send_keys(password)
    
    Login2 = driver.find_elements_by_class_name("_t38eb")
    
    Login2[0].click()

In [163]:
login_instagram(driver, 'charles.verleyen@fourcast.io','Legrosnoob007')

In [164]:
driver.get("https://www.instagram.com/"+insta_path+the_item+"/")

In [26]:
print(driver.title)

#justinbieber • Instagram photos and videos


## Send key strokes to scroll down the page

In [27]:
#press space over an over again for a while
last_how_many = -1
how_many = 0
overlap = set()
all_links = []
counter = 1

#VPN connection up
os.system('nmcli c up Trust.Zone-France')
print("VPN turning on")
sleep(15)
public_ip = get('https://api.ipify.org').text
print('The public IP address is:',public_ip)

#Start the scraping that will last until the "max_scrape" quantity or once it reaches an overlap
while len(set(all_links)) < max_scrape and len(overlap)<=9:
    
    # send a key stroke to the page to scroll down
    some_element = driver.find_element_by_tag_name("body")
    last_how_many = how_many
    for i in range(5):
        some_element.send_keys(" ")
        sleep(1)
    
    links = []
    # check if the posts on the page have already been scraped
    links = [u.find_elements_by_tag_name("a")[0].get_attribute('href')\
             .split("/")[4] for u in driver.find_elements_by_class_name("_mck9w")]
    all_links.extend(links)

    overlap = set(the_posts.keys()).intersection(set(all_links))
    
    how_many = len(all_links)
    
    print(how_many,end=" | ",flush=True)
    
    #once a multiple of 300 links have been scraped, VPN is restarted to change location and avoid being caught
    if how_many  > counter * 300:
        os.system('nmcli c down Trust.Zone-France')
        print("VPN turning off")
        sleep(15)
        os.system('nmcli c up Trust.Zone-France')
        print("VPN turning on")
        sleep(15)
        public_ip = get('https://api.ipify.org').text
        print('The public IP address is:',public_ip)
        counter += 1

os.system('nmcli c down Trust.Zone-France')
print("VPN turning off")
print("overlap: ", overlap)

VPN turning on
The public IP address is: 195.154.191.12
57 | 111 | 165 | VPN turning off
overlap:  set()


In [13]:
# print the number of links/codes found in the page and save the list in a temporary file in case everything breaks down.
print(len(all_links))
f = open(data_path+"/temp_links.json","w")
json.dump(list(set(all_links)),f)
f.close()

1506


## scrape post pages + download media objects (photos or videos)

In [14]:
def fixit(ss):
    return "".join([s for s in ss if s in ascii_letters+digits]).lower()

In [15]:
def clstr(ss):
    return "".join([s for s in ss if s in printable])

In [20]:
#function to have a visual on all the data that can be retrieved from the master page of the hashtag
def total_posts(hashtag):
    tt = requests.get("https://www.instagram.com/explore/tags/" + hashtag,timeout=5)
    tt_soup = BeautifulSoup(tt.text, "lxml")
    all_texts = [u.get_text() for u in tt_soup.find_all("script")]
    all_texts = [at for at in all_texts if "window._sharedData" in at[:50]][0]
    all_texts = all_texts.replace("window._sharedData","")
    all_texts = all_texts.replace("true","True")
    all_texts = all_texts.replace("false","False")
    all_texts = all_texts.replace("null","None")
    info = literal_eval(all_texts[3:-1])
    
    return info['entry_data']['TagPage'][0]['graphql']['hashtag']['edge_hashtag_to_media']['count']

In [21]:
#Check the total number of posts for this hashtag
print("Total posts for", the_item, "is",total_posts(the_item))

Total posts for mx5 is 921165


In [18]:
#function to have a visual on all the data that can be retrieved from a link
from ast import literal_eval
def raw_info_per_post(links):
    tt = requests.get("https://www.instagram.com/p/" + links[0],timeout=2)
    tt_soup = BeautifulSoup(tt.text, "lxml")
    all_texts = [u.get_text() for u in tt_soup.find_all("script")]
    all_texts = [at for at in all_texts if "window._sharedData" in at[:50]][0]
    all_texts = all_texts.replace("window._sharedData","")
    all_texts = all_texts.replace("true","True")
    all_texts = all_texts.replace("false","False")
    all_texts = all_texts.replace("null","None")
    info = literal_eval(all_texts[3:-1])
    
    return info

In [19]:
#Check the first link of all the links retrieved
import pprint
pp = pprint.PrettyPrinter(indent=0)
pp.pprint(raw_info_per_post(all_links))

{'activity_counts': None,
'config': {'csrf_token': 'IyqMVFslzw8W3BmGzii5iWVkFkg6zCzw', 'viewer': None},
'country_code': 'BE',
'display_properties_server_guess': {'orientation': '',
                                   'pixel_ratio': 1.5,
                                   'viewport_height': 480,
                                   'viewport_width': 360},
'entry_data': {'PostPage': [{'graphql': {'shortcode_media': {'__typename': 'GraphVideo',
                                                        'caption_is_edited': True,
                                                        'comments_disabled': False,
                                                        'dash_info': {'is_dash_eligible': False,
                                                                     'number_of_qualities': 0,
                                                                     'video_dash_manifest': None},
                                                        'dimensions': {'height': 937,
             

In [20]:
#function to extract words that follow directly a hashtag sign
def extract_hashtags(ss):
    ss = ss.replace("\n"," ")
    plist = ss.split(" ")
    tags = list(p for p in plist if len(p)>0 and p[0]=="#")
    tt = []
    for t in tags:
        tt += t.split("#")
    return list(set(list(fixit(t) for t in tt if len(t)>0)))

In [21]:
#function that will dowload the media (picture or video) from a URL
def download_object(code):
    if not os.path.isfile(media_path+"/"+the_posts[code].get("the_fname","nofile.ppp")):
        try:
            a_media_object = requests.get(the_posts[code]["object_url"],timeout=2)
            with open(media_path+"/"+the_posts[code]["the_fname"], "wb") as f:
                f.write(a_media_object.content)
            print(".",end="",flush=True)
        except:
            print("x",end="",flush=True)
    else:
        print("_",end="",flush=True)

In [22]:
# scrape a single link and retrieve data for each link. Returns in the end 
# a dictionary with all the data for this link
def scrape_single_code(a_code,sleeper=1):
    from ast import literal_eval
    good_stuff = None
    all_fine = True
    try:
        tt = requests.get("https://www.instagram.com/p/"+a_code,timeout=2)
        tt_soup = BeautifulSoup(tt.text, "lxml")
        sleeper = 2
    except:
        all_fine = False
        print("no soup for " + a_code, end="",flush=True)
        sleep(sleeper)
    
    if all_fine:
        # find all the javascript tags in the code
        all_texts = [u.get_text() for u in tt_soup.find_all("script")]

        # extract the one that starts with this particular string
        all_texts = [at for at in all_texts if "window._sharedData" in at[:50]][0]

        # fix the JavaScript syntax so that it can be transformed to Python code
        all_texts = all_texts.replace("window._sharedData","")
        all_texts = all_texts.replace("true","True")
        all_texts = all_texts.replace("false","False")
        all_texts = all_texts.replace("null","None")
        

        # extract the dict from the code and reach for the good stuff
        stuff = literal_eval(all_texts[3:-1])
        try:
            good_stuff = copy(stuff["entry_data"]["PostPage"][0]["graphql"])
        except:
            return None
        
        # add some extra to the dict
        good_stuff["scrape_ts"] = str(datetime.now())[:19]
        try:
            good_stuff["caption_tags"] = extract_hashtags(good_stuff["shortcode_media"]["edge_media_to_caption"]["edges"][0]["node"].get("text",""))
        except:
            good_stuff["caption_tags"] = []
        
        if good_stuff["shortcode_media"]["is_video"]:
            good_stuff["object_url"] = good_stuff["shortcode_media"]["video_url"]
            good_stuff["the_fname"] = a_code + ".mp4"
        else:
            good_stuff["object_url"] = good_stuff["shortcode_media"]["display_resources"][0]["src"]
            good_stuff["the_fname"] = a_code + ".jpg"       

        print(".",end="",flush=True)
    return good_stuff

In [24]:
# Master code that will iterate through all the links to get the master data (no media scraped at this point)
def scrape_codes(some_codes):
    new_stuff = {}
    sleeptime=1
    cc = 0
    
    print("VPN turning on")
    os.system('nmcli c up Trust.Zone-France')
    sleep(15)
    
    for a_code in some_codes:

        cc += 1
        if cc % 50==0: 
            print(len(new_stuff))
            
        new_stuff[a_code] = copy(scrape_single_code(a_code,sleeptime))
        
        if new_stuff[a_code]==None:
            del new_stuff[a_code]
            sleeptime += sleeptime
        else:
            sleeptime = 1
            
#VPN ON/OFF
        if cc % 300==0: 
            os.system('nmcli c down Trust.Zone-France')
            print("VPN turning off")
            sleep(15)
            os.system('nmcli c up Trust.Zone-France')
            print("VPN turning on")
            sleep(15)
            public_ip = get('https://api.ipify.org').text
            print('The public IP address is:',public_ip)
            
    return copy(new_stuff)

In [25]:
# Add the the data to "the_posts" object that keep tracks of the changes 
# in the different versions of the scraping
def update_post_storage(new_stuff):
    cc = 0
    for code in new_stuff:
        cc += 1
        if cc % 50 == 0: 
            print()
        if not code in the_posts:
            the_posts[code] = copy(new_stuff[code])
        if not "activity" in the_posts[code]:
            the_posts[code]["activity"] = []
        
        tss = [ts["scrape_ts"] for ts in the_posts[code]["activity"]]
    
        if not new_stuff[code]["scrape_ts"] in tss:
            the_posts[code]["activity"] += [{"scrape_ts":new_stuff[code]["scrape_ts"],
                                         "comments":new_stuff[code]['shortcode_media']['edge_media_to_comment']['count'],
                                         "likes":new_stuff[code]['shortcode_media']['edge_media_preview_like']['count'],
                                         "video_views":new_stuff[code]['shortcode_media'].get("video_view_count","not_a_video")}]
            print(len(the_posts[code]["activity"]),end="",flush=True)
        else:
            print("-",end="",flush=True)
    print("\nPosts in storage:",len(the_posts))

In [26]:
# scrape new post pages (not the photo itself) and add to storage
ns = scrape_codes([u for u in list(set(all_links)) if not u in the_posts])
print("\n"+str(len(ns)))
update_post_storage(ns)

VPN turning on
.................................................49
..................................................99
.......no soup for BfGKiexBd7d..........................................148
............................................no soup for BfGAo-FBWCk.....197
..................................................247
..................................................297
.VPN turning off
VPN turning on
The public IP address is: 185.128.25.43
.................................................347
..................................................397
..................................................447
..................................................497
..................................................547
..................................................597
.VPN turning off
VPN turning on
The public IP address is: 185.128.25.43
.................................................647
..................................................697
..............................................

In [30]:
#Check for links that were not scraped
a = [c for c in all_links if c not in ns]

In [31]:
a

['BfGKiexBd7d', 'BfGKiexBd7d', 'BfGAo-FBWCk']

In [32]:
# update metadata for posts in storage
def update_scraping():
    update_frequency = 24 # hours

    codes_ready_for_update = []
    for code in the_posts:
        latest_scrape = the_posts[code]["activity"][-1]["scrape_ts"]
        time_since_latest_scrape = (datetime.now()-datetime.strptime(latest_scrape,"%Y-%m-%d %H:%M:%S")).total_seconds()
        if time_since_latest_scrape > update_frequency * 3600:
            codes_ready_for_update += [code]

    if len(codes_ready_for_update)>0 and input("Do you want to update "+str(len(codes_ready_for_update))+" posts? [Y/N] ") in "Yy":
        ns = scrape_codes(codes_ready_for_update)
        update_post_storage(ns)
    else:
        print("No posts to update.")

In [33]:
update_scraping()

No posts to update.


In [34]:
# exctract comments and tags in comments for posts in the storage. Add as two attributes in the_posts.
# Also update caption tags from caption
for code in the_posts:
    
    try:
        the_posts[code]["caption_tags"] = extract_hashtags(the_posts[code]["shortcode_media"]\
                                            ["edge_media_to_caption"]["edges"][0]["node"].get("text",""))
    except:
        the_posts[code]["caption_tags"] = []
    
    comments = []
    for a in the_posts[code]['shortcode_media']['edge_media_to_comment']['edges']:
        comments += [a['node']['text']]
                        
    comments = list(set(comments))
    the_posts[code]["comm_texts"]=comments
    the_posts[code]["comm_tags"]=extract_hashtags(" ".join(comments))

In [40]:
#sanity check
the_posts['BfIjwA3Hyy5']

{'activity': [{'comments': 0,
   'likes': 43,
   'scrape_ts': '2018-02-13 16:40:40',
   'video_views': 'not_a_video'}],
 'caption_tags': ['',
  'night',
  'cool',
  'nd',
  'afternoon',
  'snow',
  'car',
  'roadster',
  'instagood',
  'mx5',
  'great',
  'japanesecar',
  'japanofinsta',
  'chillin',
  'miata',
  'mazda',
  'instagramjapan',
  'japan',
  'picoftheday',
  'photooftheday',
  'nikon',
  'thebest',
  'bringdownthegovernment',
  'in'],
 'comm_tags': [],
 'comm_texts': [],
 'object_url': 'https://instagram.fprg2-1.fna.fbcdn.net/vp/60cfb3e91116f9060e2d544bde124cad/5B119FE2/t51.2885-15/s640x640/sh0.08/e35/27578429_169525600343355_1688713121390133248_n.jpg',
 'scrape_ts': '2018-02-13 16:40:40',
 'shortcode_media': {'__typename': 'GraphImage',
  'caption_is_edited': False,
  'comments_disabled': False,
  'dimensions': {'height': 720, 'width': 1080},
  'display_resources': [{'config_height': 426,
    'config_width': 640,
    'src': 'https://instagram.fprg2-1.fna.fbcdn.net/vp/60cf

In [41]:
# download media objects (pictures or videos)
cc = 0

os.system('nmcli c up Trust.Zone-France')
print("VPN turning on")
sleep(15)
public_ip = get('https://api.ipify.org').text
print('The public IP address is:',public_ip)

for code in the_posts:
    cc += 1
    if cc % 50 == 0: print()
    download_object(code)
    
#VPN ON/OFF
    if cc % 300==0: 
        os.system('nmcli c down Trust.Zone-France')
        print("VPN turning off")
        sleep(15)
        os.system('nmcli c up Trust.Zone-France')
        print("VPN turning on")
        sleep(15)
        public_ip = get('https://api.ipify.org').text
        print('The public IP address is:',public_ip)

VPN turning on
The public IP address is: 185.128.25.35
.................................................
..................................................
..................................................
..................................................
..................................................
..................................................
.VPN turning off
VPN turning on
The public IP address is: 185.128.25.43
.................................................
..................................................
x.................................................
..................................................
..................................................
..................................................
.VPN turning off
VPN turning on
The public IP address is: 81.246.13.94
.................................................
..................................................
..................................................
........................................

In [47]:
!ls data/media_photos_mx5 -l | grep ^- | wc -l

999


## Dump

In [48]:
f = open(data_path+"/metadata.json","w")
json.dump(the_posts,f)
f.close()

In [210]:
driver.close()

## Create HTML for the Photo Grid

In [57]:
def get_total_likes(p):
    likes = 0
    for u in p['activity']:
        if u["video_views"] == "not_a_video":
            likes += u["likes"]
        else:
            likes += u["video_views"] 
    return likes

In [421]:
if deets["display"]["selected_labels"]==[]:
    selected_labels = set(all_labels)
else:
    selected_labels = set(deets["display"]["selected_labels"])
    
banned_labels = set(deets["display"]["banned_labels"])

In [58]:
if deets["display"]["selected_tags"]==[]:
    selected_tags = set(cap_tags+comm_tags)
else:
    selected_tags = set(deets["display"]["selected_tags"])
    
banned_tags = set(deets["display"]["banned_tags"])

In [59]:
print(len(the_posts),"posts in total")
the_code = ''

cc = 1
the_size = 150
include_this = True


for code in the_posts:
    include_this = True
    if cc > 50000: break
        
    ##############################################################################
    ## Start of the section where photos/videos are selected to be included or not
    if 'labels' in the_posts[code]:
        photo_labels = set(map(lambda x:x["description"], the_posts[code]["labels"]))
    else:
        photo_labels = set()
    
    if  not the_posts[code]["shortcode_media"]['is_video'] and len(all_labels)>0:
        if len(photo_labels.intersection(selected_labels))==0 or \
        len(photo_labels.intersection(banned_labels))>0: 
            include_this = False

    photo_tags =  set(the_posts[code]["caption_tags"]+the_posts[code]["caption_tags"])
    if len(cap_tags+comm_tags)>0:
        if len(photo_tags.intersection(selected_tags))==0 or \
        len(photo_tags.intersection(banned_tags))>0:
            include_this = False

    if get_total_likes(the_posts[code])>deets["display"]["max_likes"]:
        include_this = False
    
    if the_posts[code]["shortcode_media"]['is_ad']: include_this = deets["display"]["include_ads"]=="yes" and include_this
    
    if deets["display"]["include_videos"]=="none" and the_posts[code]["is_video"]: include_this = False
    if deets["display"]["include_photos"]=="none" and not the_posts[code]["is_video"]: include_this = False    
    
    # the conditions below can turn back a "False" generated in the previous statements
    if len(photo_labels)==0 and not the_posts[code]["shortcode_media"]["is_video"]: include_this = True
    if len(photo_tags)==0: include_this = True
    
    if deets["display"]["include_videos"]=="all" and the_posts[code]["is_video"]: include_this = True
    if deets["display"]["include_photos"]=="all" and not the_posts[code]["is_video"]: include_this = True
    ##################################################################################
    ## End of the section for selecting photos/videos
    ##################################################################################
    
    
    if include_this:
        cc += 1
        if not the_posts[code]["shortcode_media"]['is_video']:
        
            the_code += '<div class="imgWrap" width='+str(the_size)+' height='+str(the_size)+'>'
            the_code += '<a href="https://www.instagram.com/p/'+code+'" target="_blank">'
            the_code += '<img class="centered-and-cropped" src="../media' + "_" + prefix + "_" + the_item + "/" +the_posts[code]['the_fname']
            the_code += '" width='+str(the_size)+' height='+str(the_size)+' alt="">'
            the_code += '<p class="imgDescription">'+clstr(the_posts[code].get("caption","")[:300])+'</p>'
            the_code += '</a>'
            the_code += '</div>&nbsp;'
        
        else:
        
            the_code += '<a href="https://www.instagram.com/p/'+code+'" target="_blank">'
            the_code += '<video height="'+str(the_size)
            the_code += '" onmouseover="this.play();" onmouseout="this.pause();">'
            the_code += '<source src="../media' + "_" + prefix + "_" + the_item + "/" +the_posts[code]['the_fname'] +'" type="video/mp4">'
            the_code += '</video>&nbsp;'
            the_code += '</a>'

print("Ready to display",cc-1,"objects")

1000 posts in total
Ready to display 1000 objects


In [61]:
f = open(data_path+"/"+the_item+"_grid.html","w")
f.write('<html><head><link rel="stylesheet" href="../insta.css"></head><body>')
f.write(the_code)
f.write("</body></html>")
f.close()